In [1]:
import gym, ray
from gym import spaces
import numpy as np
from scipy.spatial import distance
# import pdb
import MultiAgentEnv as ma_env

from policy import PolicyNetwork
from ray.rllib.utils.annotations import override
from ray.rllib.models import ModelCatalog
from ray.tune.registry import register_env
from ray import tune
import ray.rllib.agents.ppo as ppo
import os, pdb
from ray.tune.logger import pretty_print
from ray.tune.logger import Logger

from typing import Dict
from ray import tune
from ray.rllib.agents.callbacks import DefaultCallbacks
from ray.rllib.env import BaseEnv
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.policy import Policy
from ray.rllib.policy.sample_batch import SampleBatch

from datetime import datetime

/home/rohit/anaconda3/envs/rlopt/lib/python3.9/site-packages/ray/autoscaler/_private/cli_logger.py:59: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
model_C = PolicyNetwork
model_H = PolicyNetwork
ModelCatalog.register_custom_model("modelC", model_C)
ModelCatalog.register_custom_model("modelH", model_H)

act_space = spaces.Box(low=-0.1,high=0.1, shape=(3,))
obs_space = spaces.Box(low=-10000,high=10000, shape=(768,))

def gen_policy(atom):
    model = "model{}".format(atom)
    config = {
        "model": {
            "custom_model": model,
        },
    }
    return (None, obs_space, act_space, config)



policies = {"policy_C": gen_policy("C"),"policy_H": gen_policy("H")}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id,  **kwargs):
    if agent_id.startswith("C"):
        pol_id = "policy_C"
    else:
        pol_id = "policy_H"
    return pol_id

def env_creator(env_config):
    return ma_env.MA_env(env_config)  # return an env instance

register_env("MA_env", env_creator)

config = ppo.DEFAULT_CONFIG.copy()

config["multiagent"] = {
        "policy_mapping_fn": policy_mapping_fn,
        "policies": policies,
        "policies_to_train": ["policy_C", "policy_H"],
    }

config["log_level"] = "WARN"
config["framework"] = "torch"
config["num_gpus"] =  int(os.environ.get("RLLIB_NUM_GPUS", "0"))
config["env_config"] =  {"atoms":["C", "H", "H", "H", "H"]}
config["rollout_fragment_length"] = 16

In [3]:
ray.init()

2021-08-07 11:28:16,972	INFO services.py:1251 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.1.65.96',
 'raylet_ip_address': '10.1.65.96',
 'redis_address': '10.1.65.96:6379',
 'object_store_address': '/tmp/ray/session_2021-08-07_11-28-15_589282_87977/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-08-07_11-28-15_589282_87977/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-08-07_11-28-15_589282_87977',
 'metrics_export_port': 60218,
 'node_id': 'b8541031a80d805f21871048581d68f39c39b45c0c19f0d88663b7ab'}

In [4]:
agent = ppo.PPOTrainer(config=config, env="MA_env")

2021-08-07 11:28:17,898	INFO trainer.py:724 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-08-07 11:28:19,595	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [5]:
# agent.restore("/home/rohit/ray_results/PPO_MA_env_2021-08-04_18-33-53ks1kndl_/checkpoint_000001/checkpoint-1")
agent.restore("/home/rohit/ray_results/PPO_MA_env_2021-08-06_18-03-5337wv3v3j/checkpoint_000021/checkpoint-21")
# agent.restore("/home/rohit/ray_results/PPO_MA_env_2021-08-06_18-37-58n4hzicpd/checkpoint_000001/checkpoint-1")

2021-08-07 11:28:19,625	INFO trainable.py:382 -- Restored on 10.1.65.96 from checkpoint: /home/rohit/ray_results/PPO_MA_env_2021-08-06_18-03-5337wv3v3j/checkpoint_000021/checkpoint-21
2021-08-07 11:28:19,626	INFO trainable.py:390 -- Current state after restoring: {'_iteration': 21, '_timesteps_total': None, '_time_total': 4056.5177500247955, '_episodes_total': 174}


In [6]:
env = ma_env.MA_env({})

In [7]:
obs = env.reset()


Reset called
forces = [8.41414131 2.30348    5.93234169 4.0053915  3.95104876] energies = -16.14551384608208 bonds = [2.01468482 1.01608474 1.4407647  1.7204536 ]


In [8]:
# agent.compute_action(obs,full_fetch=True)

In [ ]:
action = {}
# all_info = []
done = False
for i in range(100):
# while done != True:
#     pdb.set_trace()
    for agent_id, agent_obs in obs.items():
    #     print(agent_id, agent_obs)
        policy_id = config['multiagent']['policy_mapping_fn'](agent_id)
    #     action[agent_id] = agent.compute_action(agent_obs, policy_id=policy_id)
        action[agent_id] = agent.compute_single_action(agent_obs, policy_id=policy_id)
    obs, rew, done, info = env.step(action)
#     print(info)
#     all_info.append(info)

forces = [7.53434065 2.24855088 2.98785724 4.29128923 4.24102014] energies = -16.90823168873242 bonds = [1.8950999  1.05506031 1.59515906 1.45808906]
forces = [6.73078051 2.16694843 2.895206   3.69850399 4.43700998] energies = -15.918754716590055 bonds = [1.90266932 1.06411847 1.76097693 1.49448522]
forces = [8.55856781 2.06282904 5.77253785 3.5991395  4.33151851] energies = -15.772174674330508 bonds = [1.78803675 1.00623275 1.84614521 1.45469498]


In [ ]:
# action

In [ ]:
# obs = env.step(action)
# for idx, val in action.items():            
#     res = agent[atom_agent_map.index(idx)].step(val)
#     obs[idx], rew[idx], done[idx], info[idx] = res

In [ ]:
obs[2]